In [41]:
import pandas as pd
import numpy as np
import requests
from sklearn.feature_extraction.text import TfidfVectorizer         ## To vectorize the movie names
import re                                                           ## To clean text
from sklearn.metrics.pairwise import cosine_similarity              ## To calculate similarity
from ipywidgets import  widgets                                     ## To create interactive interface
from IPython.display import display                                 ## To create interactive interface

In [26]:
df = pd.read_csv(r"C:\Users\yusuf\Desktop\Workspace url\ml-25m\movies.csv")               ## Load the movie data
df.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [27]:
def clean_title(x):                                             ## Create a function
    return re.sub('[^\w ]','',x)                               ## This code removes anything except numbers,letters and blanks

In [28]:
df['clean_title']=df.title.apply(lambda x:clean_title(x.strip()))                  ## Use the function to clean the title text in each row.

In [29]:
vec = TfidfVectorizer(ngram_range=(1,2))                      ## Vectorizer converts the test into numpy arrays, it takes single words and word pairs into consideration
vec_data=vec.fit_transform(df.clean_title)                    ## Transform the cleaned text column

In [30]:
def search(query):
    query = clean_title(query)                             ## Clean the variable passed in the function
    query = vec.transform([query])                           ## Vectorize the variable   **  Only transform **
    similarity = cosine_similarity(query,vec_data).flatten()             ## Calculate the  similarity score
    locs = np.append(np.argpartition(similarity,-10)[-10:],np.argmax(similarity))      ## Find 10 indices with the highest score
    movies = df.iloc[locs][::-1].drop_duplicates()              ## Pass the indices in the movie data frame and create a new data frame.
    return movies                                          ## Return the data frame

In [31]:
search('Game of Thrones')                       ## Try the function

,movieId,title,genres,clean_title
60130,202533,Game of Thrones: The Last Watch (2019),Documentary,Game of Thrones The Last Watch 2019
59481,200950,Game of Thrones - Conquest & Rebellion (2017),Animation|Fantasy,Game of Thrones Conquest Rebellion 2017
52157,184691,Game of Death (2017),Horror|Thriller,Game of Death 2017
40248,158984,A Game of Death (1945),Action|Adventure|Thriller,A Game of Death 1945
19334,100457,Game of Death (2010),Action|Adventure|Thriller,Game of Death 2010
4336,4441,Game of Death (1978),Action,Game of Death 1978
53051,186625,Game of Aces (2016),Action|Adventure|War,Game of Aces 2016
23517,118324,Game of Werewolves (2011),Comedy|Horror,Game of Werewolves 2011
46545,172665,The Game of Truth (2013),Comedy|Drama,The Game of Truth 2013
49587,179193,Enter the Game of Death (1978),Action,Enter the Game of Death 1978


In [32]:
movie_input = widgets.Text(value = 'Toy Story',                                 ## Create a widget for an interactive interface
                           description = 'Movie Title',
                           disabled = False)
movie_list = widgets.Output()

def on_type(data):
    with movie_list:
        movie_list.clear_output()
        # display(data)
        title = data['new']
        if len(title)>3:
            display(search(title))


movie_input.observe(on_type, names='value')
display(movie_input, movie_list)

Text(value='Toy Story', description='Movie Title')

Output()

In [33]:
ratings = pd.read_csv(r"C:\Users\yusuf\Desktop\Workspace url\ml-25m\ratings.csv")             ## Load the ratings dataanalysis
ratings.head(3)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828


In [34]:
def recommendation(movie_id):
    ## Get userIds of people who liked the movie registered with the specified movie id. We can assume those users are similar users. I will refer this group as similar users to make things clear.
    similar_users = ratings[(ratings.movieId==movie_id) & (ratings.rating>4)]['userId'].unique()
    ## Collect the Ids of the other movies that similar people liked. Assume that similar people generally like similar movies.
    recs = ratings[(ratings.userId.isin(similar_users)==True) & (ratings.rating>4)]['movieId']
    ## Calculate which movie is liked how many times by similar users and divide it to the total number of the group. It shows us the percentage of people who like the movie
    recs = recs.value_counts() / len(similar_users)
    ## Filter the movies that are liked by at least %10 of the group.
    recs = recs[recs > 0.1]
    ## The data that show all users who liked the movies that the at least % 10 of the similar users also liked.
    all_=ratings[(ratings.movieId.isin(recs.index)==True) & (ratings.rating>4)]
    ## Calculate the ratio of the total population who liked the movies that the similar users liked.
    all_recs=all_['movieId'].value_counts()/len(all_['userId'].unique())
    ## Concatenate the ratio tables to see the comparison
    combined_recs = pd.concat([recs,all_recs],axis=1)
    ## Rename columns
    combined_recs.columns = ['similar','all']
    ## To calculate the score we use the percentages. If a movie is liked by similar people but not popular among the total population, it is assumed to be a better recommendation, because recommendation, in its nature, is valuable when the asker do not know about the movie. So we take the raio between the score among the similar people and the total population; the score is amplified when divided.
    combined_recs['score'] = combined_recs['similar'] / combined_recs['all']
    ## Sort the data frame by score
    combined_recs=combined_recs.sort_values('score',ascending=False)
    ## Merge scores and the movies data frames on movieId column, filter 3 columns and the first 10 rows.
    return combined_recs.merge(df,left_index=True,right_on='movieId').head(10)[['title','genres','score']]

In [35]:
movie_input_name = widgets.Text(          ## Create a text input widget
    value='Toy Story',                         ## initial value
    description = 'Movie Title:',
    disabled = False)

recommendation_list = widgets.Output()
def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title= data['new']
        if len(title)>3:
            results = search(title)
            movie_id = results.iloc[0]['movieId']
            display(recommendation(movie_id))

movie_input_name.observe(on_type,names='value')
display(movie_input_name,recommendation_list)

Text(value='Toy Story', description='Movie Title:')

Output()

In [62]:
import json
url = 'https://www.dropbox.com/s/slh5e7qzhqnjqqw/ratings.csv?dl=1'
url = 'https://www.dropbox.com/s/hjmdh3yrdck4gsd/movies.csv?dl=1'

In [63]:
# url = 'https://drive.google.com/file/d/1i8NhEUG5JIa1iQikpd1vG98CcWn8VpIR/view?usp=sharing'
# path = 'https://drive.google.com/uc?export=download&id='+url.split('/')[-2]
# re = requests.get(url)
# data = json.loads(re.text)


ratings = pd.read_csv(url)
ratings.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance


In [61]:
re.text

'movieId,title,genres\r\n1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy\r\n2,Jumanji (1995),Adventure|Children|Fantasy\r\n3,Grumpier Old Men (1995),Comedy|Romance\r\n4,Waiting to Exhale (1995),Comedy|Drama|Romance\r\n5,Father of the Bride Part II (1995),Comedy\r\n6,Heat (1995),Action|Crime|Thriller\r\n7,Sabrina (1995),Comedy|Romance\r\n8,Tom and Huck (1995),Adventure|Children\r\n9,Sudden Death (1995),Action\r\n10,GoldenEye (1995),Action|Adventure|Thriller\r\n11,"American President, The (1995)",Comedy|Drama|Romance\r\n12,Dracula: Dead and Loving It (1995),Comedy|Horror\r\n13,Balto (1995),Adventure|Animation|Children\r\n14,Nixon (1995),Drama\r\n15,Cutthroat Island (1995),Action|Adventure|Romance\r\n16,Casino (1995),Crime|Drama\r\n17,Sense and Sensibility (1995),Drama|Romance\r\n18,Four Rooms (1995),Comedy\r\n19,Ace Ventura: When Nature Calls (1995),Comedy\r\n20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller\r\n21,Get Shorty (1995),Comedy|Crime|Thriller\r\n22,Copyca